# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it`...`

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

## Technical Question / Answerer — Chat App Prototype

This notebook implements a **streaming technical Q&A assistant** with a Gradio web UI.

### Features
- **Streaming responses** — tokens appear in real-time as the model generates them
- **Model switching** — dropdown to switch between OpenRouter, OpenAI, or an offline stub model
- **Voice input (STT)** — record from your microphone; audio is transcribed to text automatically
- **Voice output (TTS)** — the assistant's reply is converted to speech and played back
- **Fallback chain** — audio features gracefully degrade depending on what's installed

### Prerequisites

**Required:**
```
pip install gradio openai python-dotenv
```

**For audio (pick what you need):**

| Package | Purpose | Install |
|---------|---------|---------|
| `SpeechRecognition` | Voice input (free, uses Google Web API) | `pip install SpeechRecognition` |
| `gTTS` | Voice output (free, uses Google Translate TTS) | `pip install gTTS` |
| `openai-whisper` | Voice input (local, more accurate, needs ~1GB) | `pip install openai-whisper` |
| `pyttsx3` | Voice output (fully offline) | `pip install pyttsx3` |

> **macOS users:** The built-in `say` command is used as a TTS fallback automatically — no install needed.

### Environment variables

Create a `.env` file in the project root with at least one API key:

```
OPENROUTER_API_KEY=sk-or-v1-...
# and/or
OPENAI_API_KEY=sk-...
```

If no keys are set, the app still runs using the **stub (offline)** model.

### How to run

Run all cells in order. The last cell launches the Gradio UI at `http://127.0.0.1:7860`.

In [ ]:
# imports
import atexit
import logging
import os
import re
import subprocess
import tempfile
import textwrap
import time
from abc import ABC, abstractmethod
from typing import Generator, Optional
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

# Optional dependencies — set to None if not installed
try:
    import whisper
except ImportError:
    whisper = None

try:
    import speech_recognition as sr
except ImportError:
    sr = None

try:
    from gtts import gTTS
except ImportError:
    gTTS = None

try:
    import pyttsx3
except ImportError:
    pyttsx3 = None

load_dotenv(override=True)


logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
log = logging.getLogger(__name__)

In [ ]:
# Constants & system prompt

MAX_TTS_CHARS = 5000
MAX_TTS_CHARS_SAY = 3000
MAX_TTS_CHARS_OPENAI = 4096
STUB_ECHO_LIMIT = 200
STUB_DELAY_SEC = 0.02
SAY_TIMEOUT_SEC = 30
CHATBOT_HEIGHT = 520

SYSTEM_PROMPT = textwrap.dedent("""\
    You are a rigorous technical assistant.

    Rules:
    1. Answer technical questions with step-by-step reasoning.
    2. State your assumptions explicitly.
    3. When asked about code, provide clear, runnable examples.
    4. Keep answers concise but correct.
    5. If you are unsure about something, say so rather than guessing.
    6. Format code blocks with the appropriate language tag.
""")

: 

In [ ]:
# Audio — Speech-to-Text (STT) and Text-to-Speech (TTS)

_whisper_model = None
_stt_backend: Optional[str] = None
_tts_backend: Optional[str] = None
_temp_audio_files: list[str] = []


def _track_temp(path: str) -> str:
    """Register a temp file for cleanup on exit."""
    _temp_audio_files.append(path)
    return path


def _cleanup_temp_audio():
    for path in _temp_audio_files:
        try:
            os.unlink(path)
        except OSError:
            pass


atexit.register(_cleanup_temp_audio)


def _strip_markdown(text: str) -> str:
    """Remove markdown formatting so TTS reads clean prose."""
    text = re.sub(r"```[\s\S]*?```", " code omitted ", text)
    text = re.sub(r"`[^`]+`", lambda m: m.group(0).strip("`"), text)
    text = re.sub(r"\*\*(.+?)\*\*", r"\1", text)
    text = re.sub(r"\*(.+?)\*", r"\1", text)
    text = re.sub(r"#{1,6}\s*", "", text)
    text = re.sub(r"\[([^\]]+)\]\([^)]+\)", r"\1", text)
    text = re.sub(r"\n{2,}", ". ", text)
    return text.strip()


def _init_stt() -> str:
    """Detect best available STT backend."""
    global _whisper_model
    if whisper is not None:
        try:
            _whisper_model = whisper.load_model("base")
            return "whisper"
        except Exception:
            pass
    if sr is not None:
        return "speech_recognition"
    return "none"


def _init_tts() -> str:
    """Detect best available TTS backend."""
    if os.getenv("OPENAI_API_KEY"):
        return "openai"
    if gTTS is not None:
        return "gtts"
    try:
        result = subprocess.run(["which", "say"], capture_output=True)
        if result.returncode == 0:
            return "macos_say"
    except FileNotFoundError:
        pass
    if pyttsx3 is not None:
        return "pyttsx3"
    return "none"


_stt_backend = _init_stt()
_tts_backend = _init_tts()
log.info("STT backend: %s | TTS backend: %s", _stt_backend, _tts_backend)


def transcribe_audio(audio_path: Optional[str]) -> str:
    """Transcribe audio file to text using the best available backend."""
    if not audio_path:
        return ""

    if _stt_backend == "whisper" and _whisper_model is not None:
        result = _whisper_model.transcribe(audio_path)
        return result.get("text", "").strip()

    if _stt_backend == "speech_recognition" and sr is not None:
        recognizer = sr.Recognizer()
        with sr.AudioFile(audio_path) as source:
            audio_data = recognizer.record(source)
        try:
            return recognizer.recognize_google(audio_data)
        except sr.UnknownValueError:
            return "[Could not understand audio]"
        except sr.RequestError as e:
            return f"[Speech recognition error: {e}]"

    return "[Audio transcription not available — install `openai-whisper` or `SpeechRecognition`]"


def text_to_speech(text: str) -> Optional[str]:
    """Convert text to an audio file. Tries each backend with fallback."""
    clean = _strip_markdown(text)[:MAX_TTS_CHARS]
    if not clean.strip():
        return None

    if _tts_backend == "openai":
        try:
            client = OpenAI()
            tmp = tempfile.NamedTemporaryFile(suffix=".mp3", delete=False)
            response = client.audio.speech.create(
                model="tts-1", voice="nova", input=clean[:MAX_TTS_CHARS_OPENAI],
            )
            response.stream_to_file(tmp.name)
            return _track_temp(tmp.name)
        except Exception:
            log.warning("OpenAI TTS failed, trying next backend", exc_info=True)

    if _tts_backend == "gtts" and gTTS is not None:
        try:
            tmp = tempfile.NamedTemporaryFile(suffix=".mp3", delete=False)
            gTTS(text=clean, lang="en").save(tmp.name)
            return _track_temp(tmp.name)
        except Exception:
            log.warning("gTTS failed, falling back to macOS say", exc_info=True)

    try:
        tmp = tempfile.NamedTemporaryFile(suffix=".aiff", delete=False)
        tmp.close()
        subprocess.run(
            ["say", "-o", tmp.name, clean[:MAX_TTS_CHARS_SAY]],
            timeout=SAY_TIMEOUT_SEC,
            check=True,
        )
        return _track_temp(tmp.name)
    except (FileNotFoundError, subprocess.SubprocessError):
        log.warning("macOS say failed", exc_info=True)

    if _tts_backend == "pyttsx3" and pyttsx3 is not None:
        try:
            tmp = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
            engine = pyttsx3.init()
            engine.save_to_file(clean, tmp.name)
            engine.runAndWait()
            return _track_temp(tmp.name)
        except Exception:
            log.warning("pyttsx3 TTS failed", exc_info=True)

    return None

2026-02-24 23:13:01,909 [INFO] STT backend: speech_recognition | TTS backend: gtts


In [ ]:
# Model abstraction — pluggable backends

class BaseModel(ABC):
    """Contract that every chat model must satisfy."""
    name: str

    @abstractmethod
    def generate(self, messages: list[dict[str, str]]) -> Generator[str, None, None]:
        """Yield streamed tokens for the given conversation."""


class StubModel(BaseModel):
    """Offline echo model — always available, no API key needed."""
    name = "stub (offline)"

    def generate(self, messages: list[dict[str, str]]) -> Generator[str, None, None]:
        last = messages[-1]["content"] if messages else ""
        reply = (
            f"**[Stub model — no API key configured]**\n\n"
            f"You said: {last[:STUB_ECHO_LIMIT]}\n\n"
            f"To enable a real model, set `OPENROUTER_API_KEY` or `OPENAI_API_KEY` in `.env`."
        )
        for word in reply.split(" "):
            yield word + " "
            time.sleep(STUB_DELAY_SEC)


class OpenAICompatibleModel(BaseModel):
    """Single class for any OpenAI-compatible endpoint (OpenAI, OpenRouter, Ollama, etc.)."""

    def __init__(self, model_id: str, display_name: str, api_key: Optional[str] = None, base_url: Optional[str] = None):
        self.name = display_name
        self._model = model_id
        self._client = OpenAI(api_key=api_key, base_url=base_url)

    def generate(self, messages: list[dict[str, str]]) -> Generator[str, None, None]:
        stream = self._client.chat.completions.create(
            model=self._model, messages=messages, stream=True,
        )
        for chunk in stream:
            delta = chunk.choices[0].delta.content if chunk.choices else None
            if delta:
                yield delta


def _build_models() -> dict[str, BaseModel]:
    models: dict[str, BaseModel] = {}
    models["stub (offline)"] = StubModel()

    if os.getenv("OPENROUTER_API_KEY"):
        or_key = os.getenv("OPENROUTER_API_KEY")
        or_url = "https://openrouter.ai/api/v1"
        models["GPT-4o-mini (OpenRouter)"] = OpenAICompatibleModel(
            "openai/gpt-4o-mini", "GPT-4o-mini (OpenRouter)", api_key=or_key, base_url=or_url,
        )
        models["Gemini 2.5 Flash (OpenRouter)"] = OpenAICompatibleModel(
            "google/gemini-2.5-flash", "Gemini 2.5 Flash (OpenRouter)", api_key=or_key, base_url=or_url,
        )

    if os.getenv("OPENAI_API_KEY"):
        models["GPT-4o-mini (OpenAI)"] = OpenAICompatibleModel("gpt-4o-mini", "GPT-4o-mini (OpenAI)")

    return models


MODELS = _build_models()
log.info("Available models: %s", list(MODELS.keys()))

2026-02-24 23:13:04,119 [INFO] Available models: ['stub (offline)', 'GPT-4o-mini (OpenRouter)', 'Gemini 2.5 Flash (OpenRouter)']


In [ ]:
# Chat logic — streaming with audio response

def _build_messages(history: list[dict[str, str]], user_message: str) -> list[dict[str, str]]:
    """Assemble the full message list from conversation history."""
    return (
        [{"role": "system", "content": SYSTEM_PROMPT}]
        + [{"role": h["role"], "content": h["content"]} for h in history]
        + [{"role": "user", "content": user_message}]
    )


def chat_respond(
    user_message: str, history: list[dict[str, str]], model_name: str,
) -> Generator[tuple[list, Optional[str]], None, None]:
    """Stream assistant response and generate optional TTS audio."""
    if not user_message.strip():
        yield history, None
        return

    model = MODELS.get(model_name) or MODELS["stub (offline)"]
    messages = _build_messages(history, user_message)

    history = history + [
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": ""},
    ]

    assistant_text = ""
    try:
        for token in model.generate(messages):
            assistant_text += token
            history[-1]["content"] = assistant_text
            yield history, None
    except Exception as e:
        log.error("Model generation failed: %s", e, exc_info=True)
        error_msg = f"\n\n⚠️ *Error: {type(e).__name__} — {e}*"
        history[-1]["content"] += error_msg
        yield history, None
        return

    try:
        audio_path = text_to_speech(assistant_text)
    except Exception:
        log.warning("TTS failed after successful generation", exc_info=True)
        audio_path = None

    yield history, audio_path


def on_audio(audio_path: Optional[str], text_input: str) -> str:
    """Transcribe audio and append to the text input."""
    transcript = transcribe_audio(audio_path)
    if transcript and not transcript.startswith("["):
        return f"{text_input} {transcript}".strip() if text_input else transcript
    return text_input or transcript

In [ ]:
# Gradio UI — launch the app

def build_ui() -> gr.Blocks:
    """Construct the Gradio interface. Separated from launch for testability."""
    model_names = list(MODELS.keys())
    default_model = model_names[1] if len(model_names) > 1 else model_names[0]

    stt_label = f"🎙 Voice input ({_stt_backend})" if _stt_backend != "none" else "🎙 Voice input (not configured)"
    tts_note = f"TTS: {_tts_backend}" if _tts_backend != "none" else "TTS: not configured"

    with gr.Blocks(title="Technical Q/A Assistant", theme=gr.themes.Soft()) as ui:
        gr.Markdown(
            "# Technical Question / Answerer\n"
            "Ask technical questions with text or voice. The assistant streams its response "
            "and can reply with audio.\n\n"
            f"*Audio status — STT: `{_stt_backend}` | TTS: `{_tts_backend}`*"
        )

        with gr.Row():
            model_dd = gr.Dropdown(choices=model_names, value=default_model, label="Model")

        with gr.Row():
            with gr.Column(scale=3):
                chatbot = gr.Chatbot(height=CHATBOT_HEIGHT, type="messages", label="Chat")
                with gr.Row():
                    txt = gr.Textbox(placeholder="Ask a technical question…", show_label=False, scale=5, lines=1)
                    send_btn = gr.Button("Send", variant="primary", scale=1)
                    clear_btn = gr.Button("Clear", variant="secondary", scale=1)
                audio_in = gr.Audio(sources=["microphone"], type="filepath", label=stt_label)
            with gr.Column(scale=1):
                gr.Markdown(f"### Audio Response\n*{tts_note}*")
                audio_out = gr.Audio(label="Assistant voice", type="filepath", interactive=False, autoplay=True)

        def submit(msg: str, hist: list, model: str):
            for updated_hist, audio in chat_respond(msg, hist, model):
                yield updated_hist, audio, ""

        send_btn.click(fn=submit, inputs=[txt, chatbot, model_dd], outputs=[chatbot, audio_out, txt])
        txt.submit(fn=submit, inputs=[txt, chatbot, model_dd], outputs=[chatbot, audio_out, txt])
        clear_btn.click(fn=lambda: ([], None, ""), outputs=[chatbot, audio_out, txt])
        audio_in.stop_recording(fn=on_audio, inputs=[audio_in, txt], outputs=[txt])
        
    return ui


app = build_ui()
app.launch(inbrowser=True)